
### **Algoritmos de clasificación con Pyspark**

**Por: José María Manzano Ortega**.


##**1. Instalación Apache Spark en Google Colab.**

En este cuaderno (notebook) se van a implementar un par de algoritmos de clasificación sobre el dataset de 10k instancias de **SUSY**, los comandos para la instalación y ejecución del entorno Spark siguen como se muestran a continuación.

In [1]:
#Primero instalamos Apache Spark con Hadoop
!wget -q https://downloads.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz

#Instalamos los paquetes de Python para trabajar con Spark
!pip install findspark #Instalamos FindSpark
!pip install pyspark   #Instalamos Spark

#Indicamos a PySpark donde está Spark
import findspark
findspark.init("spark-3.5.3-bin-hadoop3")#SPARK_HOME

#Inicializamos las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

##**2. Incoporar el SparkPackage de CatBoost en la sesión.**

Para poder usar un paquete de SparkPackages se lo tenemos que indicar al cluster. Para que se lo descargue y lo pueda ejecutar.

En este ejemplo vamos a usar el paquete CatBoost para Spark que es de los pocos que tienen tanto soporte en Python como en Scala. Para ello lo haremos al crear la sesión, usando el método config:

```
config("spark.jars.packages", "ai.catboost:catboost-spark_3.5_2.12:1.2.3")
```

Con este le indicamos que vamos a usar una bibliotecar externa en un jar de SparkPackages y dicha biblioteca es `ai.catboost:catboost-spark_3.5_2.12:1.2.3`.


In [2]:

#Creamos una sesión de Spark para poder trabajar
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Ejemplo de CatBoost en PySpark") \
    .config("spark.jars.packages", "ai.catboost:catboost-spark_3.5_2.12:1.2.3") \
    .getOrCreate()

##**3. Preparamos los datos.**

Lo primero que vamos a hacer es descargarnos los datos con los que vamos a trabajar y guardarlos en dos DataFrame: uno de entrenamiento (dfTra) y otro de test (dfTst).

In [3]:
#Nos descargamos los ficheros de datos en Google Colab
!wget -nv --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HOrM49tCLA_NqHyD_ps_cv483FPN7aWo' -O susy-10k-tra.csv
!wget -nv --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HT80d5cwU7HMi2XK8CNgxgHvxRZEZB_d' -O susy-10k-tst.csv

#Leemos los conjuntos de entrenamiento y test
dfTra = spark.read \
    .format("csv") \
    .option("inferSchema",True) \
    .option("header", True) \
    .load("susy-10k-tra.csv")
dfTst = spark.read.csv('susy-10k-tst.csv', inferSchema=True, header=True)

dfTra.show(5)
dfTst.show(5)

2024-09-26 15:10:14 URL:https://drive.usercontent.google.com/download?id=1HOrM49tCLA_NqHyD_ps_cv483FPN7aWo&export=download [3463157/3463157] -> "susy-10k-tra.csv" [1]
2024-09-26 15:10:17 URL:https://drive.usercontent.google.com/download?id=1HT80d5cwU7HMi2XK8CNgxgHvxRZEZB_d&export=download [3469204/3469204] -> "susy-10k-tst.csv" [1]
+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+-----+
|               uno|                dos|                tres|             cuatro|               cinco|               seis|              siete|                ocho|              nueve|                diez|              once|              doce|             trece|            catorce|            quince| 

  Ahora preparamos los datos para poder trabajar con ellos.

In [4]:
#Preprocesamos los datos para usarlos en un algoritmo de aprendizaje supervisado
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

#Convertimos la variable clase en entera
dfTra=dfTra.withColumn("clase",dfTra.clase.cast("Integer"))
dfTst=dfTst.withColumn("clase",dfTst.clase.cast("Integer"))

#Cambiamos el nombre de la columna donde está la variable de salida
dfTra=dfTra.withColumnRenamed("clase","label")
dfTst=dfTst.withColumnRenamed("clase","label")

#Unimos los atributos con VectorAssembler
assembler = VectorAssembler(inputCols=dfTra.columns[:-1],outputCol="features")

##**4. Ejemplo con `Decision Tree`.**

Veamos ahora un clasificador de los que vienen implementados en Apache Spark:

In [5]:
from pyspark.ml.classification import DecisionTreeClassifier

#Definimos el árbol de decisión (DT)
dt = DecisionTreeClassifier(maxDepth=4)

#Creamos una tubería, con el preprocesamiento y el DT
tuberia=Pipeline().setStages( [assembler, dt] )

#Construimos el modelo con los datos de entrenamiento
modeloDT = tuberia.fit(dfTra)

#Hacemos predicciones con el modelo aprendido (preprocesamiento + DT)
predicciones = modeloDT.transform(dfTst)

#Ya podemos mostrar la bondad del modelo
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluador = MulticlassClassificationEvaluator(metricName="accuracy")
print('Accuracy Decision Tree:', evaluador.evaluate(predicciones))

Accuracy Decision Tree: 0.7763


##**5. Ejemplo con `CatBoost`.**

De entre los algoritmos más populares que se basan en árboles de potenciación del gradiente encontramos CatBoost. Que está disponible en SparkPackages y se puede usar desde Python y Scala.


In [6]:
import catboost_spark

catb = catboost_spark.CatBoostClassifier(iterations=10)

#Creamos una tubería, con el preprocesamiento y CatBoost
tuberia=Pipeline( stages=[assembler, catb] )

#Construimos el modelo con los datos de entrenamiento
modelolcatB = tuberia.fit(dfTra)

#Hacemos predicciones con el modelo aprendido (preprocesamiento + CatBoost)
predicciones = modelolcatB.transform(dfTst)

#Ya podemos mostrar la bondad del modelo
print('Accuracy CatBoost:', evaluador.evaluate(predicciones))

Accuracy CatBoost: 0.8037
